In [2]:
import pandas as pd

In [1]:
import pandas as pd

# First, processing and preparing the emissions dataset:

# load data
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/annual_air_emissions.csv')

# filter data 
df = df[df['Gas_Type'] == 'Carbon dioxide']
cols_to_select = ['F{}'.format(year) for year in range(2010, 2022)]
df = df[cols_to_select]

# generate output
output = {
    'labels': [],
    'values': []
}
for col in df.columns.tolist():
    output['labels'].append(col[1:])
    output['values'].append(df[col].sum())
    
print(output)

# Next, the energy transition dataset:

# load data
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/energy_transition.csv')
# df.head()

# filter data
renewable_df = df[df['Energy_Type'] == "Total Renewable"]
non_renewable_df = df[df['Energy_Type'] == "Total Non-Renewable"]

cols_to_select = ['F{}'.format(year) for year in range(2000, 2021)]
renewable_df = renewable_df[cols_to_select]
non_renewable_df = non_renewable_df[cols_to_select]

# generate output
output = {
    "renewable": {
        'labels': [],
        'values': []
    },
    "non_renewable": {
        'labels': [],
        'values': []
    },
}

for col in renewable_df.columns.tolist():
    output['renewable']['labels'].append(col[1:])
    output['renewable']['values'].append(df[col].sum())
    
for col in non_renewable_df.columns.tolist():
    output['non_renewable']['labels'].append(col[1:])
    output['non_renewable']['values'].append(df[col].sum())

print(output)

# Next, process the stats on UK household waste:

# load dataset
df = pd.read_csv("/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/uk_waste_stats.csv")
# df.head()

# prepare data
df = df[df['Measure'] == 'Recycling rate']
df = df[['Year','Measure','UK']]
df.head()

# generate output
output = {
    'recycling_rate': {
        'labels': [],
        'values': []
    }
}
for index, row in df.iterrows():
    output['recycling_rate']['labels'].append(row['Year'])
    output['recycling_rate']['values'].append(row['UK'])
    
print(output)




{'labels': ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'], 'values': [347281.108599036, 357164.129203444, 362576.51395351195, 367143.053957388, 370164.63728685095, 368087.792373102, 367358.274453194, 373584.65754200297, 381289.436725764, 381536.116002269, 361909.968731441, 380874.15337320295]}
{'renewable': {'labels': ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'values': [120257439.2143567, 121250602.571417, 125835765.27341701, 130206193.17641701, 135760891.9123047, 141158011.3425107, 146349423.4272768, 152461076.0733458, 155867326.8988108, 156089953.04070678, 165781600.4476386, 171199248.54092962, 175542736.9831332, 180759547.0959541, 184764000.18295872, 188540021.9097914, 193990067.31953448, 199651070.2852374, 207063949.5988404, 210314667.7598194, 212490440.76309842]}, 'non_renewable': {'labels': ['2000', '2001

In [32]:
# load the dataset
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/processed_ni_waste_data.csv')

# initial pre-processing
df.rename(columns={'AreaName': 'area'}, inplace=True)
df.fillna(0, inplace=True)
df.replace('-', 0, inplace=True)

str_cols = ['quarter','quarter_name','fin_year','area']
for col in df.columns.tolist():
    if col in str_cols:
        pass
    else:
        df[col] = df[col].apply(lambda x: str(x).replace(',',''))
        df[col] = df[col].astype('float')

# show sample of data
df.head()

,quarter,quarter_name,fin_year,area,la_total_waste,la_reuse,la_dry_recycle,la_compost,la_energy_recovery_mixed_residual,la_energy_recovery_specific_stream,la_landfill,la_biodegradable_to_landfill,hh_total_waste,hh_reuse,hh_dry_recycle,hh_compost,hh_landfill,num_households,population
0,Q1,April to June,2006/07,Antrim,9885.0,0.0,2542.0,2353.0,0.0,0.0,4990.0,0.0,8595.0,0.0,1716.0,2353.0,4525.0,0.0,51059.0
1,Q1,April to June,2006/07,Ards,13137.0,0.0,1231.0,2191.0,0.0,0.0,9715.0,0.0,12357.0,0.0,1167.0,2191.0,8998.0,0.0,75767.0
2,Q1,April to June,2006/07,Armagh,7912.0,0.0,1443.0,1116.0,0.0,0.0,5353.0,0.0,6997.0,0.0,980.0,1116.0,4900.0,0.0,56785.0
3,Q1,April to June,2006/07,Ballymena,10072.0,0.0,1466.0,1511.0,0.0,0.0,7094.0,0.0,9502.0,0.0,1466.0,1511.0,6525.0,0.0,61447.0
4,Q1,April to June,2006/07,Ballymoney,3772.0,0.0,550.0,0.0,0.0,0.0,3222.0,0.0,3286.0,0.0,550.0,0.0,2735.0,0.0,29392.0


In [33]:
# local authority (la) waste dataset
la_df = df[['quarter','quarter_name','fin_year','area','la_total_waste','la_reuse','la_dry_recycle','la_compost','la_energy_recovery_mixed_residual','la_energy_recovery_specific_stream','la_landfill','la_biodegradable_to_landfill','num_households','population']]
la_df.head()

,quarter,quarter_name,fin_year,area,la_total_waste,la_reuse,la_dry_recycle,la_compost,la_energy_recovery_mixed_residual,la_energy_recovery_specific_stream,la_landfill,la_biodegradable_to_landfill,num_households,population
0,Q1,April to June,2006/07,Antrim,9885.0,0.0,2542.0,2353.0,0.0,0.0,4990.0,0.0,0.0,51059.0
1,Q1,April to June,2006/07,Ards,13137.0,0.0,1231.0,2191.0,0.0,0.0,9715.0,0.0,0.0,75767.0
2,Q1,April to June,2006/07,Armagh,7912.0,0.0,1443.0,1116.0,0.0,0.0,5353.0,0.0,0.0,56785.0
3,Q1,April to June,2006/07,Ballymena,10072.0,0.0,1466.0,1511.0,0.0,0.0,7094.0,0.0,0.0,61447.0
4,Q1,April to June,2006/07,Ballymoney,3772.0,0.0,550.0,0.0,0.0,0.0,3222.0,0.0,0.0,29392.0


In [37]:
# household (hh) waste dataset
hh_df = df[['quarter','quarter_name','fin_year','area','hh_total_waste','hh_reuse','hh_dry_recycle','hh_compost','hh_landfill','num_households','population']]
hh_df.head()

,quarter,quarter_name,fin_year,area,hh_total_waste,hh_reuse,hh_dry_recycle,hh_compost,hh_landfill,num_households,population
0,Q1,April to June,2006/07,Antrim,8595.0,0.0,1716.0,2353.0,4525.0,0.0,51059.0
1,Q1,April to June,2006/07,Ards,12357.0,0.0,1167.0,2191.0,8998.0,0.0,75767.0
2,Q1,April to June,2006/07,Armagh,6997.0,0.0,980.0,1116.0,4900.0,0.0,56785.0
3,Q1,April to June,2006/07,Ballymena,9502.0,0.0,1466.0,1511.0,6525.0,0.0,61447.0
4,Q1,April to June,2006/07,Ballymoney,3286.0,0.0,550.0,0.0,2735.0,0.0,29392.0


In [41]:
# PREPARE DATASETS

# prepare hh waste
hh_df['hh_recycle_rate'] = (hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']) / hh_df['hh_total_waste'] * 100
# print(hh_df['hh_recycle_rate'])

# prepare la waste
la_df['la_recycle_rate'] = (la_df['la_reuse'] + la_df['la_dry_recycle'] + la_df['la_compost']) / la_df['la_total_waste'] * 100

0       47.341478
1       27.174881
2       29.955695
3       31.330246
4       16.737675
          ...    
1327    52.581250
1328    56.465542
1329    60.634867
1330    53.006637
1331    52.913314
Name: hh_recycle_rate, Length: 1332, dtype: float64


/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m0000gn/T/ipykernel_21622/1930313435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hh_df['hh_recycle_rate'] = (hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']) / hh_df['hh_total_waste'] * 100


In [56]:
# PREPARE OUTPUT FOR KPIs

# KPI 1 - Current Period Recycle rate (%) (whole NI): 
output = {
    'kpi_1_output': {
        'title': 'Household Recycle Rate',
        'values': [],
        'labels': []
    },
    'kpi_2_output': {},
    'kpi_3_output': {},
}
for year in hh_df['fin_year'].unique():
    output['kpi_1_output']['values'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].mean()) 
    output['kpi_1_output']['labels'].append(year)

print(output)

{'kpi_1_output': {'title': 'Household Recycle Rate', 'values': [29.045953319128767, 33.11107144283889, 35.1717490704289, 36.478542211385644, 38.22951305010704, 40.90425135732721, 40.495834434694025, 41.479287370253864, 41.731075011237195, 42.1660464776332, 44.334825749951214, 48.02532365492825, 50.17396133599464, 51.8367463893321, 51.21410868241299, 50.434725847451176, 53.92189734075921], 'labels': ['2006/07', '2007/08', '2008/09', '2009/10', '2010/11', '2011/12', '2012/13', '2013/14', '2014/15', '2015/16', '2016/17', '2017/18', '2018/19', '2019/20', '2020/21', '2021/22', '2022/23']}, 'kpi_2_output': {}, 'kpi_3_output': {}}


In [ ]:
# KPI 2 - HH waste per capita (kg):
# kpi_2_output = {}
# for year in hh_df['fin_year'].unique():
